# Testing with slices of len==20 of real fasta files
First two cells, I read in files and then slice.
Second to last cell, I am trying to run it but get an error.

In [29]:
#!/usr/bin/env python3
#Biopython example for Blosum62
import sys
!{sys.executable} -m pip install 'biopython'
from Bio import pairwise2
from Bio import SeqIO
from Bio.Align import substitution_matrices
blosum62=substitution_matrices.load("BLOSUM62")
#seq1 = SeqIO.read("example_fasta_files/homo_sapiens_lactate.fasta", "fasta")
#eq2 = SeqIO.read("example_fasta_files/mus_musculus_lactate.fasta", "fasta")
seq1 = SeqIO.read("example_fasta_files/corvyllus.fasta", "fasta")
seq2 = SeqIO.read("example_fasta_files/drosophilaAE014297.fasta", "fasta")
# cast to string
a_str = str(seq1.seq)
b_str = str(seq1.seq)

In [1]:
#local alignment using Aligner: 
from Bio import Align
aligner = Align.PairwiseAligner()




AttributeError: module 'Bio.Align' has no attribute 'Pairwise'

In [30]:
print(f'{a_str[:20]}\n{b_str[20:40]}')
slice_a = a_str[:20]
slice_b = b_str[20:40]

MATLKDQLIYNLLKEEQTPQ
NKITVVGVGAVGMACAISIL


In [31]:
import numpy as np


Compare two input symbols

In [32]:
def compare(a, b, match, gap, mis_match):
    if a == b:
        return match
    elif " " == a or b == " ":
        return gap
    else:
        return mis_match

We build the matrics, the intake variable are two protein sequences, match score, mismatch score, and gap penalty. 
first the we need to initialize the matrix based on the length of two sequences. 
Then we calculate 

In [33]:
#currently the matrix is looking for minimum number, thus the scoreig
def build_matrics(seq1, seq2, match, mis_match, gap):
    a = len(seq1)
    b = len(seq2)
    
    #initialize the matrics
    S = np.zeros((a + 1, b + 1))
    
    #set 0 to the first row and the first column, or we can initialize the number from 1 to len(sequence)
    #as mentioned in https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm
    
    S[0, :] = np.fromfunction(lambda x, y: gap * (x + y), (1, b + 1), dtype=int)
    S[:, 0] = np.fromfunction(lambda x, y: gap * (x + y), (1,a + 1), dtype=int)
    
    #add empty space before the sequece.
    seq1 = " " + seq1[:]
    seq2 = " " + seq2[:]
    
    
    #longer length sequence set as row
    for i in range(1, b+1 if a>b else a+1):
        #iterate loop to find the minimum score.
        for j in range(i, b+1):
            last_score = [S[i - 1, j], S[i - 1, j - 1], S[i, j - 1]]
            change_score = [gap,compare(seq1[i],seq2[j], match, gap, mis_match),gap]

            new_score = np.add(last_score, change_score)

            S[i,j] = max(new_score)
            # print(S[i,c])
        for j in range(i+1, a+1):

            last_score = [S[j-1,i], S[j-1, i-1], S[j, i - 1]]
            change_score = [gap,compare(seq1[j],seq2[i], match, gap, mis_match),gap]
            new_score = np.add(last_score, change_score)
            S[j,i] = max(new_score)
    return S


In [39]:
def trace_back(seq1, seq2, match, mismatch, gap, S, current_x, current_y, S1, S2, t1, t2, score, ts):
    
    if current_x == 0 and current_y == 0:
        #print("reached here")
        S1.append(seq1[1] + t1[:])
        S2.append(seq2[1] + t2[:])
        score.append(ts)
        return
    if current_x>0 and current_y>0:
        if S[current_x,current_y]-S[current_x-1,current_y-1] == compare(seq1[current_x], seq2[current_y], match, mismatch, gap):
            #print(current_x, current_y, t1)
            ts = ts + S[current_x,current_y]
            trace_back(seq1, seq2, match, mismatch, gap, S, current_x-1, current_y-1, S1, S2, seq1[current_x] + t1[:], seq2[current_y] + t2[:], score, ts)
    if current_y>0:
        if S[current_x,current_y]-S[current_x,current_y-1] == gap:
            #print(current_x, current_y, t1)
            ts = ts + S[current_x,current_y]
            trace_back(seq1, seq2, match, mismatch, gap, S, current_x, current_y-1, S1, S2, "-" + t1[:], seq2[current_y] + t2[:], score, ts)
    if current_x>0:
        if S[current_x,current_y]-S[current_x-1,current_y] == gap:
            #print(current_x, current_y, t1)
            ts = ts + S[current_x,current_y]
            trace_back(seq1, seq2, match, mismatch, gap, S, current_x-1, current_y, S1, S2, seq1[current_x]+t1[:], "-"+t2[:], score, ts)


In [45]:

F=build_matrics(slice_a, slice_b, 3,-1,-1)
S1 = []
S2 = []
score=[]
ts=0
t1 = ""
t2 = ""
trace_back(" "+slice_a, " "+slice_b, 3, -1, -1, F, F.shape[0]-1, F.shape[1]-1, S1, S2, t1, t2, score, ts)
#print(f'S1:\n{S1}') # all the seq1's in a list
#print(f'S2:\n{S2}') # all seq2's in list
print(S1[score.index(max(score))])
print(S2[score.index(max(score))])
print(f'F:\n{F}') # the matrix

M-MAT-------LKDQLIYNLLKEEQTPQ
NNKITVVGVGAVGMACAI-SIL-------
F:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.
  -14. -15. -16. -17. -18. -19. -20.]
 [ -1.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12.  -9.
  -10. -11. -12. -13. -14. -15. -16.]
 [ -2.  -2.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.  -6.  -7.  -8.  -9.
   -6.  -7.  -8.  -9. -10. -11. -12.]
 [ -3.  -3.  -3.  -3.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.
   -7.  -7.  -8.  -9. -10. -11. -12.]
 [ -4.  -4.  -4.  -4.  -1.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.
   -8.  -8.  -8.  -9. -10. -11.  -8.]
 [ -5.  -5.  -1.  -2.  -2.  -2.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9.
   -9.  -9.  -9.  -9. -10. -11.  -9.]
 [ -6.  -6.  -2.  -2.  -3.  -3.  -3.  -3.  -4.  -5.  -6.  -7.  -8.  -9.
  -10. -10. -10. -10. -10. -11. -10.]
 [ -7.  -7.  -3.  -3.  -3.  -4.  -4.  -4.  -4.  -5.  -6.  -7.  -8.  -9.
  -10. -11. -11. -11. -11. -11. -11.]
 [ -8.  -8.  -4.  -4.  -4.  -4.  -5.  -5.  -5.  -5.  -6. 

In [23]:
F = build_matrics("ATGGC","AGG",3,-1,-1)
S1 = []
S2 = []
t1 = ""
t2 = ""
trace_back(" ATGGC", " AGG", 3, -1, -1, F, F.shape[0]-1, F.shape[1]-1, S1, S2, t1, t2)
print(f'S1:\n{S1}')
print(f'S2:\n{S2}')
print(f'F:\n{F}')
print(type(S1), type(S2), type(F))

5 3 
4 3 C
3 2 GC
2 1 GGC
1 1 TGGC
reached here
S1:
['AATGGC']
S2:
['AA-GG-']
F:
[[ 0. -1. -2. -3.]
 [-1.  3.  2.  1.]
 [-2.  2.  2.  1.]
 [-3.  1.  5.  5.]
 [-4.  0.  4.  8.]
 [-5. -1.  3.  7.]]
<class 'list'> <class 'list'> <class 'numpy.ndarray'>
